In [1]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from PIL import Image
import cv2
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, InputLayer, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

In [2]:
csv_file = "../data/train_labels.csv"
csv_file_2 = "../data/train_labels_2.csv"

df_1 = pd.read_csv(csv_file)
df_2 = pd.read_csv(csv_file_2)
df = pd.concat([df_1, df_2], ignore_index=True, )
df.rename(columns={'Bags used': 'bags_used', 'Pothole number': 'pothole_number'}, inplace=True)


In [3]:
annotations_folder = "../data/train_annotations/"
annotations_folder_2 = "../data/train_annotations_2/"

In [4]:

# List to store the extracted data
data = []

# Loop through each annotation file
for file_name in os.listdir(annotations_folder):
    if file_name.endswith('.txt'):
        image_name = file_name.replace('.txt', '')  # Assuming image has the same name as the annotation file
        
        # Open and read the annotation file
        with open(os.path.join(annotations_folder, file_name), 'r') as file:
            for line in file:
                # Split the line into its components
                class_id, x_center, y_center, width, height = line.strip().split()
                
                # Append the data to the list
                data.append({
                    'image_name': image_name,
                    'class_id': int(class_id),
                    'x_center': float(x_center),
                    'y_center': float(y_center),
                    'width': float(width),
                    'height': float(height)
                })

# Convert the list to a DataFrame
anno = pd.DataFrame(data)


In [5]:
data = []

# Loop through each annotation file
for file_name in os.listdir(annotations_folder_2):
    if file_name.endswith('.txt'):
        image_name = file_name.replace('.txt', '')  # Assuming image has the same name as the annotation file
        
        # Open and read the annotation file
        with open(os.path.join(annotations_folder_2, file_name), 'r') as file:
            for line in file:
                # Split the line into its components
                class_id, x_center, y_center, width, height = line.strip().split()
                
                # Append the data to the list
                data.append({
                    'image_name': image_name,
                    'class_id': int(class_id),
                    'x_center': float(x_center),
                    'y_center': float(y_center),
                    'width': float(width),
                    'height': float(height)
                })

# Convert the list to a DataFrame
anno_2 = pd.DataFrame(data)

# Display the DataFrame
print(anno_2.head())

  image_name  class_id  x_center  y_center     width    height
0       p930         0  0.449839  0.499142  0.799943  0.998284
1       p930         1  0.550746  0.162950  0.071658  0.325900
2       p924         1  0.777287  0.467410  0.445397  0.049743
3       p924         0  0.519282  0.546312  0.917953  0.704974
4        p58         0  0.481026  0.474271  0.483542  0.667238


In [6]:
annotations = pd.concat([anno, anno_2], ignore_index=True, )
annotations['pothole_number'] = annotations['image_name'].str.replace('p', '').astype(int)
annotations

,image_name,class_id,x_center,y_center,width,height,pothole_number
0,p1336,1,0.634134,0.553173,0.562607,0.094340,1336
1,p1108,0,0.540652,0.533448,0.560892,0.463122,1108
2,p1108,1,0.576672,0.540309,0.564322,0.174957,1108
3,p1120,1,0.387135,0.484563,0.596913,0.053173,1120
4,p1120,0,0.439451,0.510292,0.372213,0.313894,1120
...,...,...,...,...,...,...,...
1716,p1663,1,0.455288,0.472034,0.727885,0.073593,1663
1717,p55,0,0.461758,0.451115,0.923516,0.356775,55
1718,p55,1,0.190507,0.435678,0.334832,0.027444,55
1719,p929,0,0.528169,0.550600,0.782471,0.874786,929


In [7]:
df[df['pothole_number'] == 929]

,pothole_number,Bags used
263,929,1.0
833,929,1.0


In [8]:
df = df.drop_duplicates()
df_annotations = annotations.merge(df,on='pothole_number', how="inner", suffixes=('', '_df'))
df_annotations['img_file'] = 'p' + df_annotations['pothole_number'].astype(str) + '.jpg'

In [9]:
df_annotations

,image_name,class_id,x_center,y_center,width,height,pothole_number,Bags used,img_file
0,p1336,1,0.634134,0.553173,0.562607,0.094340,1336,0.25,p1336.jpg
1,p1108,0,0.540652,0.533448,0.560892,0.463122,1108,0.50,p1108.jpg
2,p1108,1,0.576672,0.540309,0.564322,0.174957,1108,0.50,p1108.jpg
3,p1120,1,0.387135,0.484563,0.596913,0.053173,1120,0.25,p1120.jpg
4,p1120,0,0.439451,0.510292,0.372213,0.313894,1120,0.25,p1120.jpg
...,...,...,...,...,...,...,...,...,...
1514,p1663,1,0.455288,0.472034,0.727885,0.073593,1663,1.00,p1663.jpg
1515,p55,0,0.461758,0.451115,0.923516,0.356775,55,2.00,p55.jpg
1516,p55,1,0.190507,0.435678,0.334832,0.027444,55,2.00,p55.jpg
1517,p929,0,0.528169,0.550600,0.782471,0.874786,929,1.00,p929.jpg


In [10]:
valid = [] 
for file in df_annotations['img_file']:
    name = '../data/train_images/' + file 
    name_2 = '../data/train_images_2/' + file 
    if os.path.exists(name):
        valid.append(file)
    elif os.path.exists(name_2):
        valid.append(file)

In [11]:
len(valid)

1519

In [12]:
valid_df = df_annotations[df_annotations['img_file'].isin(valid)]
valid_df

,image_name,class_id,x_center,y_center,width,height,pothole_number,Bags used,img_file
0,p1336,1,0.634134,0.553173,0.562607,0.094340,1336,0.25,p1336.jpg
1,p1108,0,0.540652,0.533448,0.560892,0.463122,1108,0.50,p1108.jpg
2,p1108,1,0.576672,0.540309,0.564322,0.174957,1108,0.50,p1108.jpg
3,p1120,1,0.387135,0.484563,0.596913,0.053173,1120,0.25,p1120.jpg
4,p1120,0,0.439451,0.510292,0.372213,0.313894,1120,0.25,p1120.jpg
...,...,...,...,...,...,...,...,...,...
1514,p1663,1,0.455288,0.472034,0.727885,0.073593,1663,1.00,p1663.jpg
1515,p55,0,0.461758,0.451115,0.923516,0.356775,55,2.00,p55.jpg
1516,p55,1,0.190507,0.435678,0.334832,0.027444,55,2.00,p55.jpg
1517,p929,0,0.528169,0.550600,0.782471,0.874786,929,1.00,p929.jpg


In [13]:
def yolo_to_bbox(x_center, y_center, width, height, img_width, img_height):
    x_min = (x_center - width / 2) * img_width
    x_max = (x_center + width / 2) * img_width
    y_min = (y_center - height / 2) * img_height
    y_max = (y_center + height / 2) * img_height
    return x_min, y_min, x_max, y_max

# Calculate the diagonal length of the bounding box for `class_id` 1
def calculate_stick_size(row, img_width, img_height):
    if (row['class_id'] != 1):
        return 0
    x_min, y_min, x_max, y_max = yolo_to_bbox(
        row['x_center'], row['y_center'], row['width'], row['height'], img_width, img_height
    )
    return np.sqrt((x_max - x_min) ** 2 + (y_max - y_min) ** 2)

# Example function to load an image and get its dimensions
def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size

# Add a column with stick sizes
df_annotations['stick_size'] = df_annotations.apply(
    lambda row: calculate_stick_size(row, *get_image_size(f'../data/train_all/{row["img_file"]}')),
    axis=1
)

In [14]:
df_stick = df_annotations

In [15]:
df_annotations['stick_size'] = df_annotations.groupby('pothole_number')['stick_size'].transform(
    lambda x: x.replace(0, x.max())
)

# If stick_size is still zero, populate it with the mean of non-zero stick_size values
mean_stick_size = df_annotations['stick_size'].replace(0, np.nan).mean()
df_annotations['stick_size'] = df_annotations['stick_size'].replace(0, mean_stick_size)



In [16]:
df_annotations['has_pothole'] = df_annotations.groupby('pothole_number')['class_id'].transform(lambda x: (x == 0).any())


In [17]:
df_annotations = df_annotations.reset_index(drop=True)
df_annotations['stick_size'].idxmax()


1275

In [18]:
idx_max = df_annotations['stick_size'].idxmax()
if idx_max in df_annotations.index:
    row_max = df_annotations.loc[idx_max]
    print(row_max)
else:
    print(f"Index {idx_max} not found in DataFrame.")

image_name             p1601
class_id                   0
x_center            0.427885
y_center                 0.5
width               0.848077
height              0.997321
pothole_number          1601
Bags used                0.5
img_file           p1601.jpg
stick_size        842.645863
has_pothole             True
Name: 1275, dtype: object


In [19]:
stick_min = df_annotations['stick_size'].mean()
df_annotations['scale_factor'] = (400 / df_annotations['stick_size'])

# Convert scale_factor to integer type if needed
#df_annotations['scale_factor'] = df_annotations['scale_factor'].astype(int)
df_annotations

,image_name,class_id,x_center,y_center,width,height,pothole_number,Bags used,img_file,stick_size,has_pothole,scale_factor
0,p1336,1,0.634134,0.553173,0.562607,0.094340,1336,0.25,p1336.jpg,215.634563,False,1.854990
1,p1108,0,0.540652,0.533448,0.560892,0.463122,1108,0.50,p1108.jpg,223.330280,True,1.791069
2,p1108,1,0.576672,0.540309,0.564322,0.174957,1108,0.50,p1108.jpg,223.330280,True,1.791069
3,p1120,1,0.387135,0.484563,0.596913,0.053173,1120,0.25,p1120.jpg,226.526572,True,1.765797
4,p1120,0,0.439451,0.510292,0.372213,0.313894,1120,0.25,p1120.jpg,226.526572,True,1.765797
...,...,...,...,...,...,...,...,...,...,...,...,...
1514,p1663,1,0.455288,0.472034,0.727885,0.073593,1663,1.00,p1663.jpg,690.079562,True,0.579643
1515,p55,0,0.461758,0.451115,0.923516,0.356775,55,2.00,p55.jpg,117.200459,True,3.412956
1516,p55,1,0.190507,0.435678,0.334832,0.027444,55,2.00,p55.jpg,117.200459,True,3.412956
1517,p929,0,0.528169,0.550600,0.782471,0.874786,929,1.00,p929.jpg,309.195260,True,1.293681


In [20]:
def resize_image(image_path, scale_factor, output_path):
    with Image.open(image_path) as img:
        # Calculate new size
        width, height = img.size
        new_size = (int(width * scale_factor), int(height * scale_factor))
        
        # Resize and save image
        img_resized = img.resize(new_size, Image.LANCZOS)
        img_resized.save(output_path)  # Save resized image

# Define a target size for the stick
for index, row in df_annotations.iterrows():
    image_path = f"../data/train_all/{row['img_file']}"
    output_path = f"../data/cropped_images/{row['img_file']}"
    scale_factor = row['scale_factor']
    
    resize_image(image_path, scale_factor, output_path)

# Resize all images to match the target size of the stick



In [21]:
def resize_and_pad_image(image_path, output_path, target_size=(512, 512), max_image_size=None):
    with Image.open(image_path) as img:
        # Calculate scaling factor based on the largest image
        if max_image_size is None:
            max_image_size = max(img.size)
        
        scale_factor = target_size[0] / max_image_size
        
        # Resize the image while keeping aspect ratio
        new_size = (int(img.width * scale_factor), int(img.height * scale_factor))
        img_resized = img.resize(new_size, Image.LANCZOS)
        
        # Create a new image with a black background
        new_img = Image.new("RGB", target_size, (0, 0, 0))
        
        # Calculate padding
        padding_x = (target_size[0] - new_size[0]) // 2
        padding_y = (target_size[1] - new_size[1]) // 2
        
        # Paste the resized image onto the black background
        new_img.paste(img_resized, (padding_x, padding_y))
        
        # Save the new image
        new_img.save(output_path)

# Example usage
image_paths = []
for index, row in df_annotations.iterrows():
    image_paths.append(f"../data/cropped_images/{row['img_file']}")
output_folder = "../data/padded_images/"
os.makedirs(output_folder, exist_ok=True)

# Find the largest image size
max_image_size = max([max(Image.open(image_path).size) for image_path in image_paths])
# Resize and pad each image
for image_path in image_paths:
    output_path = os.path.join(output_folder, f"resized_{os.path.basename(image_path)}")
    resize_and_pad_image(image_path, output_path, target_size=(512, 512), max_image_size=max_image_size)


In [32]:
def add_randomness_to_colors_tf(image):
    color_shift_range = tf.random.uniform([], 0, 15, dtype=tf.float32)

    # Convert the image to a float32 tensor and normalize the values to [0, 1]
    img_tensor = tf.image.convert_image_dtype(image, tf.float32)

    # Generate random shifts for each color channel
    random_shift = tf.random.uniform(tf.shape(img_tensor), -color_shift_range/255.0, color_shift_range/255.0)

    # Add the random shift and clip values to stay within valid range [0, 1]
    img_tensor = tf.clip_by_value(img_tensor + random_shift, 0.0, 1.0)

    # Convert back to the original data type
    img_tensor = tf.image.convert_image_dtype(img_tensor, tf.float32)

    return img_tensor

In [33]:
class Augment(tf.keras.layers.Layer):
    def __init__(self, contrast_range=[0.4, 1.5], 
                 brightness_delta=[-0.3, 0.3],
                 hue_delta=[-0.1, 0.1],
                 jpeg_qual = [40,100],
                 **kwargs):
        super(Augment, self).__init__(**kwargs)
        self.contrast_range = contrast_range
        self.brightness_delta = brightness_delta
        self.hue_delta = hue_delta
        self.jpeg_qual = jpeg_qual
    
    def ensure_rank_4(self, images):
        """Ensure that the images tensor has rank 4."""
        if len(images.shape) == 3:
            # If the image is rank 3 (height, width, channels), add a batch dimension
            images = tf.expand_dims(images, axis=0)
        elif len(images.shape) == 5:
            # If the image is rank 5 (batch_size, height, width, channels, extra), squeeze out the extra dimension
            images = tf.squeeze(images, axis=-1)
        return images
    
    def call(self, images, training=None):
        if not training:
            return images
        
        images = self.ensure_rank_4(images)
        
        contrast = np.random.uniform(
            self.contrast_range[0], self.contrast_range[1])
        brightness = np.random.uniform(
            self.brightness_delta[0], self.brightness_delta[1])
        hue = np.random.uniform(
            self.hue_delta[0], self.hue_delta[1])
        jpeg = random.randint(
            self.jpeg_qual[0], self.jpeg_qual[1])
        flip_u = random.randint(0,2)
        flip_l = random.randint(0,2)
        
        contr = random.randint(0,1)
        bright = random.randint(0,1)

        ad_hue = random.randint(0,1)
        j_q = random.randint(0,2)

        rand_col = random.randint(0,1)
        soft = random.randint(0,2)

        rot = random.randint(0, 2)

        if (contr == 1) :images = tf.image.adjust_contrast(images, contrast)
        if (bright == 1) :images = tf.image.adjust_brightness(images, brightness)
        images = tf.clip_by_value(images, 0, 1)
        if (ad_hue == 1) :images = tf.image.adjust_hue(images, hue)
        #if (j_q== 1) :images = tf.image.adjust_jpeg_quality(images, jpeg, dct_method='')
        if (rand_col == 1) :images = add_randomness_to_colors_tf(images)
        #if (soft == 1) :images = soften_edges_tf(images)
        if (flip_l == 1) : images = tf.image.flip_left_right(images)
        if (flip_u == 1) : images = tf.image.flip_up_down(images)
        if (rot == 1) : images =tf.image.rot90(images)

        return images

In [34]:

# Define image dimensions
img_width, img_height = 256, 256

# Preprocess images and load into arrays
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0, 1]
    return img_array

# Load and preprocess images
X = np.array([preprocess_image('../data/padded_images/resized_' + file) for file in valid_df['img_file']])
y = valid_df['Bags used '].values
scaler = MinMaxScaler()

# Split data into train and test sets
X_train, X_test, y_train_un, y_test_un = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = scaler.fit_transform(y_train_un.reshape(-1, 1)).flatten()
y_test = scaler.transform(y_test_un.reshape(-1, 1)).flatten()


In [35]:
len(y_train)

1215

In [36]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    patience=50,          # Stop after 5 epochs of no improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best loss
)

In [42]:
augment_layer = Augment()

model = Sequential([
    InputLayer(input_shape=(256, 256, 3)),  # Replace with your input image size
    augment_layer,  # Add the augmentation layer here
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='linear')  # Regression output for predicting 'bags_used'
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Summary of the model
model.summary()


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augment_6 (Augment)             │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 254, 254, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 127, 127, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 125, 125, 16)   │         9,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 62, 62, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 61504)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │     1,968,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,979,729 (7.55 MB)

 Trainable params: 1,979,729 (7.55 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
history = model.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test), batch_size=15, callbacks=[early_stopping])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {mae}')

Epoch 1/150


2024-08-15 23:14:58.530009: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:933] Skipping loop optimization for Merge node with control input: StatefulPartitionedCall/sequential_3_1/augment_3_1/rot90/Assert/AssertGuard/branch_executed/_51


81/81 ━━━━━━━━━━━━━━━━━━━━ 10s 122ms/step - loss: 0.0366 - mean_absolute_error: 0.0814 - val_loss: 0.0032 - val_mean_absolute_error: 0.0339
Epoch 2/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 10s 120ms/step - loss: 0.0104 - mean_absolute_error: 0.0443 - val_loss: 0.0032 - val_mean_absolute_error: 0.0360
Epoch 3/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 10s 121ms/step - loss: 0.0089 - mean_absolute_error: 0.0438 - val_loss: 0.0029 - val_mean_absolute_error: 0.0335
Epoch 4/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 10s 121ms/step - loss: 0.0037 - mean_absolute_error: 0.0353 - val_loss: 0.0032 - val_mean_absolute_error: 0.0387
Epoch 5/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 10s 123ms/step - loss: 0.0077 - mean_absolute_error: 0.0414 - val_loss: 0.0033 - val_mean_absolute_error: 0.0365
Epoch 6/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 10s 124ms/step - loss: 0.0086 - mean_absolute_error: 0.0433 - val_loss: 0.0031 - val_mean_absolute_error: 0.0329
Epoch 7/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 11s 130ms/step - loss: 0.0070 - mean_absolute_error: 0.0406 -

In [237]:
# Save the weights
# model.save('../model_weights/CNN_padded_images.keras')


In [39]:
metric = tf.metrics.R2Score()
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("R2 Score: ", r2)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step
R2 Score:  0.24078618418058073


In [283]:
# Assuming y_true are the actual values and y_pred are the predictions from your model

def round_to_nearest_25(values):
    rounded_values = np.round(values * 4) / 4
    return rounded_values

metric = tf.metrics.R2Score()
y_pred = model.predict(X_test)
y_in = scaler.inverse_transform(y_pred)
y_rounded = round_to_nearest_25(y_in)
r2 = r2_score(y_test_un, y_rounded)
r2

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step


0.41395695597000903